In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlite3 # library for working with sqlite database

### GETTING TABLES

In [ ]:
# Create connection
conn = sqlite3.connect("./data/labevents_balanced.db")
conn2 = sqlite3.connect("./data/lab_events.db") # keep this

# Get dataframe
lab_events_df = pd.read_sql("""SELECT * FROM labevents""", conn)
lab_items_df = pd.read_sql("""SELECT * FROM lab_items""", conn2)

In [ ]:
# Convert to numeric data types
lab_events_df["SUBJECT_ID"] = pd.to_numeric(lab_events_df["SUBJECT_ID"])
lab_events_df["HADM_ID"] = pd.to_numeric(lab_events_df["HADM_ID"]).astype(int)
lab_events_df["VALUENUM"] = pd.to_numeric(lab_events_df["VALUENUM"])

### CLEAN DATA

In [ ]:
# Remove rows where VALUENUM is nan
lab_events_df = lab_events_df.loc[(lab_events_df["VALUENUM"]).notna(), :]

# Remove negative values
#lab_events_df = lab_events_df.loc[lab_events_df["VALUENUM"] >= 0,:]

# Merge with labels for laboratory measurements
lab_events_df = lab_events_df.merge(lab_items_df[['ITEMID','LABEL']], left_on='ITEMID', right_on='ITEMID')

In [ ]:
# 
lab_events_df.head()

### TRUTH LABELS

In [ ]:
# Column 1: SUBJECT_ID
# Column 2: TRUTH
# Column 3: HADM_ID
truth_array = np.loadtxt("csv/TRUTH.csv", delimiter=',', skiprows=1)

# Create a dataframe with truth values
truth_df = pd.DataFrame(truth_array, columns=['SUBJECT_ID', 'TRUTH', 'HADM_ID'])

# Convert to ints
truth_df["SUBJECT_ID"] = truth_df["SUBJECT_ID"].astype(int)
truth_df["TRUTH"] = truth_df["TRUTH"].astype(int)
truth_df["HADM_ID"] = truth_df["HADM_ID"].astype(int)

### GET RELEVANT ADMISSIONS

In [ ]:
# Only interested in lab events corresponding to the relevant admissions
relevant_lab_events_df = lab_events_df.merge(truth_df, on='HADM_ID')

In [ ]:
# Get relevant truth labels
hadm_df = pd.DataFrame(lab_events_df["HADM_ID"].unique(), columns={'HADM_ID'})

# Get updated truth_df
relevant_truth_df = hadm_df.merge(truth_df, how='inner', on='HADM_ID')

### GET FEATURES

In [ ]:
# Take the mean of every measurement for every subject
avg_df = relevant_lab_events_df.groupby(['HADM_ID', 'LABEL'], as_index = False)['VALUENUM'].median()
avg_df = avg_df.rename(index=str, columns={"VALUENUM": "AVG"})

In [ ]:
meas_type = np.unique(lab_events_df["LABEL"].get_values())

design_mat_df = relevant_truth_df
for meas in meas_type:
    # Get avg of current measurement
    meas_avg_df = avg_df.loc[avg_df["LABEL"] == meas, ["HADM_ID", "AVG"]]
    
    # Rename
    rename_as = meas + "_AVG"
    meas_avg_df = meas_avg_df.rename(index=str, columns={"AVG": rename_as})
    
    # Merge
    design_mat_df = design_mat_df.merge(meas_avg_df, how='left', on='HADM_ID')


#### Missing Values

In [ ]:
# Create additional columns for missing values
#for meas in meas_type:
#    # Get current column
#    curr_meas = design_mat_df.loc[:, meas + "_AVG"]
    
#    # Add column for missing values
#    design_mat_df[meas + "_AVG_missing"] = curr_meas.isna().astype(int)

In [ ]:
# Median Imputation
design_mat_df = design_mat_df.fillna(design_mat_df.median())

In [ ]:
design_mat_df.head()

In [ ]:
# Get X and y values
X = design_mat_df.iloc[:, 3:len(design_mat_df.columns)].get_values()
y = design_mat_df.TRUTH.get_values()

### CLASSIFICATION

In [ ]:
from sklearn.model_selection import train_test_split

# make train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.33, random_state = 0)

In [ ]:
from sklearn.preprocessing import normalize

#Normalize
X_train = normalize(X_train, norm='max', axis=0)
X_test = normalize(X_test, norm='max', axis=0)

In [ ]:
from sklearn.svm import SVC

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
y_pred = svm.decision_function(X_test)

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
AUC = metrics.auc(fpr, tpr)
print(AUC)